In [1]:
import tensorflow as tf

2022-10-22 16:29:56.291616: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/gurobi952/linux64/lib
2022-10-22 16:29:56.291638: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from pathlib import Path

lane_nav_model_path = Path("lane_navigation/model/lane_navigation_final.h5")

In [4]:
lane_nav_model = tf.keras.models.load_model(lane_nav_model_path)

In [12]:
import numpy as np

In [15]:
def representative_data_gen():
    train_images = np.zeros((100, 66, 200, 3), dtype=np.float32)
    for input_value in (
        tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(1)
    ):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(lane_nav_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [16]:
lane_nav_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp0ig8gapq/assets


INFO:tensorflow:Assets written to: /tmp/tmp0ig8gapq/assets
2022-10-22 16:44:07.137995: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-10-22 16:44:07.138021: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-10-22 16:44:07.138151: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp0ig8gapq
2022-10-22 16:44:07.139524: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-10-22 16:44:07.139535: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmp0ig8gapq
2022-10-22 16:44:07.143922: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-10-22 16:44:07.191445: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmp0ig8gapq
2022-10-22 16:44:07.204668: I tensorflow/cc/saved_model/loader.cc:283] SavedMode

In [17]:
with tf.io.gfile.GFile("lane_navigation/model/lane_navigation_final.tflite", "wb") as f:
    f.write(lane_nav_tflite_model)

In [18]:
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter(model_path="lane_navigation/model/lane_navigation_final.tflite")

In [19]:
import time

In [20]:
interpreter.allocate_tensors()

for _ in range(5):
    st = time.perf_counter()
    interpreter.invoke()
    dur = (time.perf_counter() - st) * 1000
    print(dur)
    

19.231311976909637
1.9459547474980354
1.9005900248885155
1.9058799371123314
1.894078217446804


In [22]:
from pycoral.utils.edgetpu import make_interpreter

interpreter = make_interpreter("lane_navigation/model/lane_navigation_final_edgetpu.tflite")
interpreter.allocate_tensors()

for _ in range(5):
    st = time.perf_counter()
    interpreter.invoke()
    dur = (time.perf_counter() - st) * 1000
    print(dur)
    

1.4542993158102036
1.1226944625377655
1.1065136641263962
1.1104326695203781
1.1090077459812164
